In [29]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# Function to split the dataset and scale features
def split_scalar(indep_X, dep_Y):
    X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size=0.25, random_state=0)
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    return X_train, X_test, y_train, y_test, sc

# Function to compute confusion matrix, accuracy, and classification report
def cm_prediction(classifier, X_test, y_test):
    y_pred = classifier.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    return classifier, accuracy, report, cm

# Classification functions
def logistic(X_train, y_train, X_test, y_test):
    classifier = LogisticRegression(random_state=0)
    classifier.fit(X_train, y_train)
    return cm_prediction(classifier, X_test, y_test)

def svm_linear(X_train, y_train, X_test, y_test):
    classifier = SVC(kernel='linear', random_state=0)
    classifier.fit(X_train, y_train)
    return cm_prediction(classifier, X_test, y_test)

def svm_nl(X_train, y_train, X_test, y_test):
    classifier = SVC(kernel='rbf', random_state=0)
    classifier.fit(X_train, y_train)
    return cm_prediction(classifier, X_test, y_test)

def naive_bayes(X_train, y_train, X_test, y_test):
    classifier = GaussianNB()
    classifier.fit(X_train, y_train)
    return cm_prediction(classifier, X_test, y_test)

def knn(X_train, y_train, X_test, y_test):
    classifier = KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=2)
    classifier.fit(X_train, y_train)
    return cm_prediction(classifier, X_test, y_test)

def decision_tree(X_train, y_train, X_test, y_test):
    classifier = DecisionTreeClassifier(criterion='entropy', random_state=0)
    classifier.fit(X_train, y_train)
    return cm_prediction(classifier, X_test, y_test)

def random_forest(X_train, y_train, X_test, y_test):
    classifier = RandomForestClassifier(n_estimators=10, criterion='entropy', random_state=0)
    classifier.fit(X_train, y_train)
    return cm_prediction(classifier, X_test, y_test)

# Load and preprocess dataset
dataset = pd.read_csv("CKD.csv")
df2 = pd.get_dummies(dataset, drop_first=True)
indep_X = df2.drop('classification_yes', axis=1)
dep_Y = df2['classification_yes']

# Feature selection using PCA
X_train, X_test, y_train, y_test, sc = split_scalar(indep_X, dep_Y)
pca = PCA(n_components=2)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

important_features = pd.DataFrame(pca.components_, columns=indep_X.columns, index=[f'PCA{i+1}' for i in range(pca.n_components)])
print(important_features)

# Initialize lists to store accuracies
acclog = []
accsvml = []
accsvmnl = []
accknn = []
accnav = []
accdes = []
accrf = []

# Logistic Regression
classifier, Accuracy, report, cm = logistic(X_train, y_train, X_test, y_test)
acclog.append(Accuracy)

# SVM Linear
classifier, Accuracy, report, cm = svm_linear(X_train, y_train, X_test, y_test)
accsvml.append(Accuracy)

# SVM Non-linear
classifier, Accuracy, report, cm = svm_nl(X_train, y_train, X_test, y_test)
accsvmnl.append(Accuracy)

# KNN
classifier, Accuracy, report, cm = knn(X_train, y_train, X_test, y_test)
accknn.append(Accuracy)

# Naive Bayes
classifier, Accuracy, report, cm = naive_bayes(X_train, y_train, X_test, y_test)
accnav.append(Accuracy)

# Decision Tree
classifier, Accuracy, report, cm = decision_tree(X_train, y_train, X_test, y_test)
accdes.append(Accuracy)

# Random Forest
classifier, Accuracy, report, cm = random_forest(X_train, y_train, X_test, y_test)
accrf.append(Accuracy)

# Compile results
results = [acclog[0], accsvml[0], accsvmnl[0], accknn[0], accnav[0], accdes[0], accrf[0]]
result_df = pd.DataFrame([results], index=['PCA'], columns=['Logistic', 'SVMl', 'SVMnl', 'KNN', 'Naive', 'Decision', 'Random'])

# Print results
print(result_df)


           age        bp        al        su       bgr        bu        sc  \
PCA1  0.119040  0.144238  0.259443  0.142802  0.171878  0.251258  0.189035   
PCA2 -0.092081  0.007455 -0.252001 -0.305408 -0.304225  0.273697  0.414356   

           sod       pot      hrmo  ...  rbc_normal  pc_normal  pcc_present  \
PCA1 -0.164306  0.056310 -0.311192  ...   -0.150379  -0.224535     0.153310   
PCA2 -0.319495  0.125112 -0.133517  ...    0.143276   0.250029    -0.251657   

      ba_present   htn_yes    dm_yes   cad_yes  appet_yes    pe_yes   ane_yes  
PCA1    0.133237  0.262261  0.240650  0.145627  -0.193558  0.199535  0.193938  
PCA2   -0.212064  0.047471 -0.076347  0.039064  -0.028453  0.001142  0.184286  

[2 rows x 27 columns]
     Logistic  SVMl  SVMnl  KNN  Naive  Decision  Random
PCA      0.99  0.99    1.0  1.0    1.0      0.99    0.99
